In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from datetime import timedelta
import gc

In [2]:
pd.set_option("display.max_columns", 300)

In [3]:
teams = json.loads(open("/Users/chienan/Pycon/github/SportLottery/nbastat/teamId_mpt").read())

In [4]:
df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_2012to2019.pkl").sort_values("Game_ID").reset_index(drop=True)

In [5]:
df.WL = df.WL.replace({"W":1,"L":0})

In [6]:
lastNgames = 10 

In [7]:
# Aggregate Data
df_ = pd.DataFrame()
for key, team_name in zip(teams.keys(),teams.values()):
    df_i = df[df.Team_ID==int(key)].reset_index(drop=True)
    df_team = pd.DataFrame()
    for times in range(1,lastNgames+1):
        mean_lst = []
        sum_lst = []
           
        for time in range(1, times+1):
            mean_lst.append(df_i.shift(time).loc[:,"FGM":"PTS"].values)
            sum_lst.append(df_i.shift(time).loc[:,"WL":"WL"].values)
        df_team = pd.concat([df_team,
                             pd.DataFrame(np.mean(np.array(mean_lst),axis=0),
                                          columns=["LAST_"+str(times)+"_GAME_"+col for col in df_i.loc[:,"FGM":"PTS"].columns]),
                             pd.DataFrame(np.sum(np.array(sum_lst),axis=0),
                                          columns=["LAST_"+str(times)+"_GAME_WL"])],axis=1)
        
    df_team = pd.concat([df_i.loc[:,"Team_ID":"MATCHUP"],df_team],axis=1)
    df_ = pd.concat([df_, df_team],axis=0).reset_index(drop=True)
    print("finish %s"%team_name)

finish Atlanta Hawks
finish Boston Celtics
finish Brooklyn Nets
finish Charlotte Hornets
finish Chicago Bulls
finish Cleveland Cavaliers
finish Dallas Mavericks
finish Denver Nuggets
finish Detroit Pistons
finish Golden State Warriors
finish Houston Rockets
finish Indiana Pacers
finish Los Angeles Clippers
finish Los Angeles Lakers
finish Memphis Grizzlies
finish Miami Heat
finish Milwaukee Bucks
finish Minnesota Timberwolves
finish New Orleans Pelicans
finish New York Knicks
finish Oklahoma City Thunder
finish Orlando Magic
finish Philadelphia 76ers
finish Phoenix Suns
finish Portland Trail Blazers
finish Sacramento Kings
finish San Antonio Spurs
finish Toronto Raptors
finish Utah Jazz
finish Washington Wizards


In [8]:
df_.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_LAST_10_GAEMS_AGG_2012to2019.pkl")

# Team Data Combine

In [9]:
teams = json.loads(open("/Users/chienan/Pycon/github/SportLottery/nbastat/teamId_mpt").read())

In [10]:
# Aggregate Data
df_ = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_LAST_10_GAEMS_AGG_2012to2019.pkl")

In [11]:
len(df_[["Team_ID","Game_ID"]].drop_duplicates())

18160

In [12]:
plus_minus = pd.concat([pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl"),
                        pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")],
                       axis=0).reset_index(drop=True)

In [13]:
plus_minus = plus_minus[['GAME_ID', 'TEAM_ID', 'GAME_DATE', 'PLUS_MINUS']]

In [14]:
plus_minus.head()

,GAME_ID,TEAM_ID,GAME_DATE,PLUS_MINUS
0,0021100001,1610612738,2011-12-25,-2.0
1,0021100002,1610612748,2011-12-25,11.0
2,0021100003,1610612741,2011-12-25,1.0
3,0021100004,1610612753,2011-12-25,-8.0
4,0021100005,1610612746,2011-12-25,19.0


In [15]:
lastNgames = 10 

In [16]:
# Aggregate Data
plus_minus_ = pd.DataFrame()
for key, team_name in zip(teams.keys(),teams.values()):
    df_i = plus_minus[plus_minus.TEAM_ID==int(key)].reset_index(drop=True)
    df_team = pd.DataFrame()
    for times in range(1,lastNgames+1):
        mean_lst = []

        
        for time in range(1, times+1):
            mean_lst.append(df_i.shift(time).loc[:,"PLUS_MINUS":"PLUS_MINUS"].values)
            
        df_team = pd.concat([df_team,pd.DataFrame(np.mean(np.array(mean_lst),axis=0),
                                                  columns=["LAST_"+str(times)+"_GAME_PLUS_MINUS"])],axis=1)
    df_team = pd.concat([df_i.loc[:,"GAME_ID":"TEAM_ID"],df_team],axis=1)
    plus_minus_ = pd.concat([plus_minus_, df_team],axis=0).reset_index(drop=True)
    print("finish %s"%team_name)

finish Atlanta Hawks
finish Boston Celtics
finish Brooklyn Nets
finish Charlotte Hornets
finish Chicago Bulls
finish Cleveland Cavaliers
finish Dallas Mavericks
finish Denver Nuggets
finish Detroit Pistons
finish Golden State Warriors
finish Houston Rockets
finish Indiana Pacers
finish Los Angeles Clippers
finish Los Angeles Lakers
finish Memphis Grizzlies
finish Miami Heat
finish Milwaukee Bucks
finish Minnesota Timberwolves
finish New Orleans Pelicans
finish New York Knicks
finish Oklahoma City Thunder
finish Orlando Magic
finish Philadelphia 76ers
finish Phoenix Suns
finish Portland Trail Blazers
finish Sacramento Kings
finish San Antonio Spurs
finish Toronto Raptors
finish Utah Jazz
finish Washington Wizards


In [17]:
# Home Team Data
nba_home_df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")
nba_home_df = nba_home_df[['TEAM_ID','GAME_ID','PLUS_MINUS']]

# Away Team Data
nba_away_df = pd.read_pickle('/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl')
nba_away_df = nba_away_df[['TEAM_ID','GAME_ID']]

# Join Two Table
nba_df = nba_home_df.merge(nba_away_df, on='GAME_ID', suffixes=['_HOME','_AWAY'],how='inner')
nba_df = nba_df[['GAME_ID','TEAM_ID_HOME','TEAM_ID_AWAY','PLUS_MINUS']]
nba_df.head()

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS
0,0021100001,1610612752,1610612738,2.0
1,0021100002,1610612742,1610612748,-11.0
2,0021100003,1610612747,1610612741,-1.0
3,0021100004,1610612760,1610612753,8.0
4,0021100005,1610612744,1610612746,-19.0


In [18]:
nba_df = nba_df.merge(df_,left_on=['TEAM_ID_HOME','GAME_ID'],right_on=['Team_ID','Game_ID'],how='inner')
nba_df = nba_df.merge(df_,left_on=['TEAM_ID_AWAY','GAME_ID'],right_on=['Team_ID','Game_ID'],
                      suffixes=['_H','_A'],how='inner')
nba_df['GAME_DATE']=nba_df['GAME_DATE_H']

In [19]:
for col in [col for col in nba_df.columns if ("Team_" in col)|("Game_ID" in col)|("MATCHUP" in col)|("GAME_DATE_" in col)]:
    del nba_df[col]

In [20]:
gc.collect()

296

In [21]:
nba_df.GAME_DATE = nba_df.GAME_DATE.apply(lambda dt : datetime.strptime(dt, "%b %d, %Y"))

In [22]:
nba_df = nba_df.merge(plus_minus_,left_on=['TEAM_ID_HOME','GAME_ID'],right_on=['TEAM_ID','GAME_ID'],how='inner')
nba_df = nba_df.merge(plus_minus_,left_on=['TEAM_ID_AWAY','GAME_ID'],right_on=['TEAM_ID','GAME_ID'],
                      suffixes=['_H','_A'],how='inner')

In [23]:
nba_df.tail()

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS,LAST_1_GAME_FGM_H,LAST_1_GAME_FGA_H,LAST_1_GAME_FG_PCT_H,LAST_1_GAME_FG3M_H,LAST_1_GAME_FG3A_H,LAST_1_GAME_FG3_PCT_H,LAST_1_GAME_FTM_H,LAST_1_GAME_FTA_H,LAST_1_GAME_FT_PCT_H,LAST_1_GAME_OREB_H,LAST_1_GAME_DREB_H,LAST_1_GAME_REB_H,LAST_1_GAME_AST_H,LAST_1_GAME_STL_H,LAST_1_GAME_BLK_H,LAST_1_GAME_TOV_H,LAST_1_GAME_PF_H,LAST_1_GAME_PTS_H,LAST_1_GAME_WL_H,LAST_2_GAME_FGM_H,LAST_2_GAME_FGA_H,LAST_2_GAME_FG_PCT_H,LAST_2_GAME_FG3M_H,LAST_2_GAME_FG3A_H,LAST_2_GAME_FG3_PCT_H,LAST_2_GAME_FTM_H,LAST_2_GAME_FTA_H,LAST_2_GAME_FT_PCT_H,LAST_2_GAME_OREB_H,LAST_2_GAME_DREB_H,LAST_2_GAME_REB_H,LAST_2_GAME_AST_H,LAST_2_GAME_STL_H,LAST_2_GAME_BLK_H,LAST_2_GAME_TOV_H,LAST_2_GAME_PF_H,LAST_2_GAME_PTS_H,LAST_2_GAME_WL_H,LAST_3_GAME_FGM_H,LAST_3_GAME_FGA_H,LAST_3_GAME_FG_PCT_H,LAST_3_GAME_FG3M_H,LAST_3_GAME_FG3A_H,LAST_3_GAME_FG3_PCT_H,LAST_3_GAME_FTM_H,LAST_3_GAME_FTA_H,LAST_3_GAME_FT_PCT_H,LAST_3_GAME_OREB_H,LAST_3_GAME_DREB_H,LAST_3_GAME_REB_H,LAST_3_GAME_AST_H,LAST_3_GAME_STL_H,LAST_3_GAME_BLK_H,LAST_3_GAME_TOV_H,LAST_3_GAME_PF_H,LAST_3_GAME_PTS_H,LAST_3_GAME_WL_H,LAST_4_GAME_FGM_H,LAST_4_GAME_FGA_H,LAST_4_GAME_FG_PCT_H,LAST_4_GAME_FG3M_H,LAST_4_GAME_FG3A_H,LAST_4_GAME_FG3_PCT_H,LAST_4_GAME_FTM_H,LAST_4_GAME_FTA_H,LAST_4_GAME_FT_PCT_H,LAST_4_GAME_OREB_H,LAST_4_GAME_DREB_H,LAST_4_GAME_REB_H,LAST_4_GAME_AST_H,LAST_4_GAME_STL_H,LAST_4_GAME_BLK_H,LAST_4_GAME_TOV_H,LAST_4_GAME_PF_H,LAST_4_GAME_PTS_H,LAST_4_GAME_WL_H,LAST_5_GAME_FGM_H,LAST_5_GAME_FGA_H,LAST_5_GAME_FG_PCT_H,LAST_5_GAME_FG3M_H,LAST_5_GAME_FG3A_H,LAST_5_GAME_FG3_PCT_H,LAST_5_GAME_FTM_H,LAST_5_GAME_FTA_H,LAST_5_GAME_FT_PCT_H,LAST_5_GAME_OREB_H,LAST_5_GAME_DREB_H,LAST_5_GAME_REB_H,LAST_5_GAME_AST_H,LAST_5_GAME_STL_H,LAST_5_GAME_BLK_H,LAST_5_GAME_TOV_H,LAST_5_GAME_PF_H,LAST_5_GAME_PTS_H,LAST_5_GAME_WL_H,LAST_6_GAME_FGM_H,LAST_6_GAME_FGA_H,LAST_6_GAME_FG_PCT_H,LAST_6_GAME_FG3M_H,LAST_6_GAME_FG3A_H,LAST_6_GAME_FG3_PCT_H,LAST_6_GAME_FTM_H,LAST_6_GAME_FTA_H,LAST_6_GAME_FT_PCT_H,LAST_6_GAME_OREB_H,LAST_6_GAME_DREB_H,LAST_6_GAME_REB_H,LAST_6_GAME_AST_H,LAST_6_GAME_STL_H,LAST_6_GAME_BLK_H,LAST_6_GAME_TOV_H,LAST_6_GAME_PF_H,LAST_6_GAME_PTS_H,LAST_6_GAME_WL_H,LAST_7_GAME_FGM_H,LAST_7_GAME_FGA_H,LAST_7_GAME_FG_PCT_H,LAST_7_GAME_FG3M_H,LAST_7_GAME_FG3A_H,LAST_7_GAME_FG3_PCT_H,LAST_7_GAME_FTM_H,LAST_7_GAME_FTA_H,LAST_7_GAME_FT_PCT_H,LAST_7_GAME_OREB_H,LAST_7_GAME_DREB_H,LAST_7_GAME_REB_H,LAST_7_GAME_AST_H,LAST_7_GAME_STL_H,LAST_7_GAME_BLK_H,LAST_7_GAME_TOV_H,LAST_7_GAME_PF_H,LAST_7_GAME_PTS_H,LAST_7_GAME_WL_H,LAST_8_GAME_FGM_H,LAST_8_GAME_FGA_H,LAST_8_GAME_FG_PCT_H,LAST_8_GAME_FG3M_H,LAST_8_GAME_FG3A_H,LAST_8_GAME_FG3_PCT_H,LAST_8_GAME_FTM_H,LAST_8_GAME_FTA_H,LAST_8_GAME_FT_PCT_H,LAST_8_GAME_OREB_H,LAST_8_GAME_DREB_H,LAST_8_GAME_REB_H,LAST_8_GAME_AST_H,...,LAST_4_GAME_FTM_A,LAST_4_GAME_FTA_A,LAST_4_GAME_FT_PCT_A,LAST_4_GAME_OREB_A,LAST_4_GAME_DREB_A,LAST_4_GAME_REB_A,LAST_4_GAME_AST_A,LAST_4_GAME_STL_A,LAST_4_GAME_BLK_A,LAST_4_GAME_TOV_A,LAST_4_GAME_PF_A,LAST_4_GAME_PTS_A,LAST_4_GAME_WL_A,LAST_5_GAME_FGM_A,LAST_5_GAME_FGA_A,LAST_5_GAME_FG_PCT_A,LAST_5_GAME_FG3M_A,LAST_5_GAME_FG3A_A,LAST_5_GAME_FG3_PCT_A,LAST_5_GAME_FTM_A,LAST_5_GAME_FTA_A,LAST_5_GAME_FT_PCT_A,LAST_5_GAME_OREB_A,LAST_5_GAME_DREB_A,LAST_5_GAME_REB_A,LAST_5_GAME_AST_A,LAST_5_GAME_STL_A,LAST_5_GAME_BLK_A,LAST_5_GAME_TOV_A,LAST_5_GAME_PF_A,LAST_5_GAME_PTS_A,LAST_5_GAME_WL_A,LAST_6_GAME_FGM_A,LAST_6_GAME_FGA_A,LAST_6_GAME_FG_PCT_A,LAST_6_GAME_FG3M_A,LAST_6_GAME_FG3A_A,LAST_6_GAME_FG3_PCT_A,LAST_6_GAME_FTM_A,LAST_6_GAME_FTA_A,LAST_6_GAME_FT_PCT_A,LAST_6_GAME_OREB_A,LAST_6_GAME_DREB_A,LAST_6_GAME_REB_A,LAST_6_GAME_AST_A,LAST_6_GAME_STL_A,LAST_6_GAME_BLK_A,LAST_6_GAME_TOV_A,LAST_6_GAME_PF_A,LAST_6_GAME_PTS_A,LAST_6_GAME_WL_A,LAST_7_GAME_FGM_A,LAST_7_GAME_FGA_A,LAST_7_GAME_FG_PCT_A,LAST_7_GAME_FG3M_A,LAST_7_GAME_FG3A_A,LAST_7_GAME_FG3_PCT_A,LAST_7_GAME_FTM_A,LAST_7_GAME_FTA_A,LAST_7_GAME_FT_PCT_A,LAST_7_GAME_OREB_A,LAST_7_GAME_DREB_A,LAST_7_GAME_REB_A,LAST_7_GAME_AST_A,LAST_7_GAME_STL_A,LAST_7_GAME_B

# Player Data Combine

In [24]:
playerBoxScoreAdv = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_2012to2019.pkl")

In [25]:
playerBoxScoreAdv = playerBoxScoreAdv[~playerBoxScoreAdv.MIN.isnull()]
playerBoxScoreAdv['MIN'] = playerBoxScoreAdv.MIN.apply(lambda x : float(x.split(':')[0]) + float(x.split(':')[1])/60.0)

In [26]:
playerBoxScoreAdv = playerBoxScoreAdv.loc[:,['GAME_ID','PLAYER_ID','TEAM_ID','MIN', 'OFF_RATING','DEF_RATING', 
                                             'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO','OREB_PCT', 'DREB_PCT',
                                             'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT','USG_PCT', 'PACE', 'PIE']]

In [27]:
playerBoxScoreAdv = playerBoxScoreAdv.fillna(0)

In [28]:
playerBoxScoreAdv["TEAM_ID"] = playerBoxScoreAdv["TEAM_ID"].astype(str)
playerBoxScoreAdv["PLAYER_ID"] = playerBoxScoreAdv["PLAYER_ID"].astype(str)

In [29]:
playerBoxScoreAdv = playerBoxScoreAdv.sort_index()

In [30]:
player_id = playerBoxScoreAdv.PLAYER_ID.unique()

In [31]:
playerBoxScoreAdv.index = [playerBoxScoreAdv.PLAYER_ID.values,playerBoxScoreAdv.GAME_ID.values]
playerBoxScoreAdv = playerBoxScoreAdv.sort_index()

In [32]:
player_id

array(['708', '2562', '979', ..., '1629312', '1629061', '1627395'],
      dtype=object)

In [33]:
# Aggregate Data
df_ = pd.DataFrame()
for player in player_id:
    player_i = playerBoxScoreAdv.loc[player]
    df_player = pd.DataFrame()
    
    for times in range(1,lastNgames+1):
        mean_lst = []
        for time in range(1, times+1):
            mean_lst.append(player_i.shift(time).loc[:,"MIN":].values)
        df_player = pd.concat([df_player,
                               pd.DataFrame(np.mean(np.array(mean_lst),axis=0),
                                            columns=["LAST_"+str(times)+"_GAME_PLAYER_"+col for col in player_i.loc[:,"MIN":].columns])
                              ],axis=1)
        
    df_player = pd.concat([player_i.loc[:,"GAME_ID":"TEAM_ID"].reset_index(drop=True),df_player],axis=1)
    df_ = pd.concat([df_, df_player],axis=0).reset_index(drop=True)

In [34]:
len(df_)

191473

In [35]:
df_.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_LAST_10_GAEMS_2012to2019.pkl")

# Add Player Data to Team

In [36]:
# aggregate data
df_ = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_LAST_10_GAEMS_2012to2019.pkl")

In [37]:
playerBoxScoreAdv = playerBoxScoreAdv.reset_index(drop=True)

In [38]:
playerBoxScoreAdv = playerBoxScoreAdv.sort_values(["PLAYER_ID","GAME_ID"]).reset_index(drop=True)

In [39]:
game_shift = playerBoxScoreAdv.groupby(["PLAYER_ID"])["GAME_ID"].shift().to_frame("GAME_ID_LAST")

In [40]:
game_shift = pd.concat([game_shift,playerBoxScoreAdv[["GAME_ID","PLAYER_ID"]]],axis=1)

In [41]:
df_["PLAYER_ID"] = df_.PLAYER_ID.astype(str)

In [42]:
game_shift = game_shift.merge(df_, left_on=["GAME_ID_LAST","PLAYER_ID"], right_on=["GAME_ID","PLAYER_ID"],suffixes=["","_LAST"], how="left")

In [43]:
game_shift.drop(["GAME_ID_LAST"],axis=1,inplace=True)

In [44]:
game_shift = game_shift[~game_shift.TEAM_ID.isna()].reset_index(drop=True)

In [45]:
game_shift = game_shift.fillna(0)

In [46]:
game_shift["TEAM_ID"] = game_shift.TEAM_ID.astype(int)
game_shift["TEAM_ID"] = game_shift.TEAM_ID.astype(str)

In [47]:
game_shift = game_shift.sort_values(["GAME_ID","TEAM_ID","PLAYER_ID"]).reset_index(drop=True)

In [48]:
game_shift["PLAYER_MIN_RATIO"] = game_shift.groupby(["GAME_ID","TEAM_ID"]).apply(lambda x : x["LAST_1_GAME_PLAYER_MIN"]/sum(x["LAST_1_GAME_PLAYER_MIN"])).values

In [49]:
game_shift = game_shift[~game_shift.PLAYER_MIN_RATIO.isna()].reset_index(drop=True)

In [50]:
game_shift.loc[:,"LAST_1_GAME_PLAYER_MIN":] = game_shift.loc[:,"LAST_1_GAME_PLAYER_MIN":].apply(lambda x : (x * x["LAST_1_GAME_PLAYER_MIN"])/240.0,axis=1)

In [51]:
game_shift = game_shift.groupby(["GAME_ID","TEAM_ID"]).sum().reset_index()

In [52]:
len(game_shift)

19340

# Combine Player and Team infomation

In [53]:
nba_df = nba_df[~nba_df.LAST_10_GAME_PLUS_MINUS_A.isna()].reset_index(drop=True)

In [54]:
len(nba_df)

8779

In [55]:
len(game_shift)

19340

In [56]:
nba_df["TEAM_ID_HOME"] = nba_df["TEAM_ID_HOME"].astype(str)
nba_df["TEAM_ID_AWAY"] = nba_df["TEAM_ID_AWAY"].astype(str)
game_shift["TEAM_ID"] = game_shift["TEAM_ID"].astype(str)

In [57]:
game_shift = (nba_df[["GAME_ID", "TEAM_ID_HOME", "TEAM_ID_AWAY"]].merge(game_shift,left_on=["GAME_ID","TEAM_ID_HOME"],right_on=["GAME_ID","TEAM_ID"],how="left")
              .merge(game_shift,left_on=["GAME_ID","TEAM_ID_AWAY"],right_on=["GAME_ID","TEAM_ID"],how="left",suffixes=['_H','_A']))

In [58]:
game_shift.drop(["TEAM_ID_H","TEAM_ID_A","PLAYER_MIN_RATIO_A","PLAYER_MIN_RATIO_H"],axis=1,inplace=True)

In [59]:
gc.collect()

91

In [60]:
nba_df = nba_df.merge(game_shift, on=["GAME_ID", "TEAM_ID_HOME", "TEAM_ID_AWAY"],how="inner" )

In [61]:
nba_df.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_AGG_2012TO2019.pkl")

In [62]:
nba_df

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS,LAST_1_GAME_FGM_H,LAST_1_GAME_FGA_H,LAST_1_GAME_FG_PCT_H,LAST_1_GAME_FG3M_H,LAST_1_GAME_FG3A_H,LAST_1_GAME_FG3_PCT_H,LAST_1_GAME_FTM_H,LAST_1_GAME_FTA_H,LAST_1_GAME_FT_PCT_H,LAST_1_GAME_OREB_H,LAST_1_GAME_DREB_H,LAST_1_GAME_REB_H,LAST_1_GAME_AST_H,LAST_1_GAME_STL_H,LAST_1_GAME_BLK_H,LAST_1_GAME_TOV_H,LAST_1_GAME_PF_H,LAST_1_GAME_PTS_H,LAST_1_GAME_WL_H,LAST_2_GAME_FGM_H,LAST_2_GAME_FGA_H,LAST_2_GAME_FG_PCT_H,LAST_2_GAME_FG3M_H,LAST_2_GAME_FG3A_H,LAST_2_GAME_FG3_PCT_H,LAST_2_GAME_FTM_H,LAST_2_GAME_FTA_H,LAST_2_GAME_FT_PCT_H,LAST_2_GAME_OREB_H,LAST_2_GAME_DREB_H,LAST_2_GAME_REB_H,LAST_2_GAME_AST_H,LAST_2_GAME_STL_H,LAST_2_GAME_BLK_H,LAST_2_GAME_TOV_H,LAST_2_GAME_PF_H,LAST_2_GAME_PTS_H,LAST_2_GAME_WL_H,LAST_3_GAME_FGM_H,LAST_3_GAME_FGA_H,LAST_3_GAME_FG_PCT_H,LAST_3_GAME_FG3M_H,LAST_3_GAME_FG3A_H,LAST_3_GAME_FG3_PCT_H,LAST_3_GAME_FTM_H,LAST_3_GAME_FTA_H,LAST_3_GAME_FT_PCT_H,LAST_3_GAME_OREB_H,LAST_3_GAME_DREB_H,LAST_3_GAME_REB_H,LAST_3_GAME_AST_H,LAST_3_GAME_STL_H,LAST_3_GAME_BLK_H,LAST_3_GAME_TOV_H,LAST_3_GAME_PF_H,LAST_3_GAME_PTS_H,LAST_3_GAME_WL_H,LAST_4_GAME_FGM_H,LAST_4_GAME_FGA_H,LAST_4_GAME_FG_PCT_H,LAST_4_GAME_FG3M_H,LAST_4_GAME_FG3A_H,LAST_4_GAME_FG3_PCT_H,LAST_4_GAME_FTM_H,LAST_4_GAME_FTA_H,LAST_4_GAME_FT_PCT_H,LAST_4_GAME_OREB_H,LAST_4_GAME_DREB_H,LAST_4_GAME_REB_H,LAST_4_GAME_AST_H,LAST_4_GAME_STL_H,LAST_4_GAME_BLK_H,LAST_4_GAME_TOV_H,LAST_4_GAME_PF_H,LAST_4_GAME_PTS_H,LAST_4_GAME_WL_H,LAST_5_GAME_FGM_H,LAST_5_GAME_FGA_H,LAST_5_GAME_FG_PCT_H,LAST_5_GAME_FG3M_H,LAST_5_GAME_FG3A_H,LAST_5_GAME_FG3_PCT_H,LAST_5_GAME_FTM_H,LAST_5_GAME_FTA_H,LAST_5_GAME_FT_PCT_H,LAST_5_GAME_OREB_H,LAST_5_GAME_DREB_H,LAST_5_GAME_REB_H,LAST_5_GAME_AST_H,LAST_5_GAME_STL_H,LAST_5_GAME_BLK_H,LAST_5_GAME_TOV_H,LAST_5_GAME_PF_H,LAST_5_GAME_PTS_H,LAST_5_GAME_WL_H,LAST_6_GAME_FGM_H,LAST_6_GAME_FGA_H,LAST_6_GAME_FG_PCT_H,LAST_6_GAME_FG3M_H,LAST_6_GAME_FG3A_H,LAST_6_GAME_FG3_PCT_H,LAST_6_GAME_FTM_H,LAST_6_GAME_FTA_H,LAST_6_GAME_FT_PCT_H,LAST_6_GAME_OREB_H,LAST_6_GAME_DREB_H,LAST_6_GAME_REB_H,LAST_6_GAME_AST_H,LAST_6_GAME_STL_H,LAST_6_GAME_BLK_H,LAST_6_GAME_TOV_H,LAST_6_GAME_PF_H,LAST_6_GAME_PTS_H,LAST_6_GAME_WL_H,LAST_7_GAME_FGM_H,LAST_7_GAME_FGA_H,LAST_7_GAME_FG_PCT_H,LAST_7_GAME_FG3M_H,LAST_7_GAME_FG3A_H,LAST_7_GAME_FG3_PCT_H,LAST_7_GAME_FTM_H,LAST_7_GAME_FTA_H,LAST_7_GAME_FT_PCT_H,LAST_7_GAME_OREB_H,LAST_7_GAME_DREB_H,LAST_7_GAME_REB_H,LAST_7_GAME_AST_H,LAST_7_GAME_STL_H,LAST_7_GAME_BLK_H,LAST_7_GAME_TOV_H,LAST_7_GAME_PF_H,LAST_7_GAME_PTS_H,LAST_7_GAME_WL_H,LAST_8_GAME_FGM_H,LAST_8_GAME_FGA_H,LAST_8_GAME_FG_PCT_H,LAST_8_GAME_FG3M_H,LAST_8_GAME_FG3A_H,LAST_8_GAME_FG3_PCT_H,LAST_8_GAME_FTM_H,LAST_8_GAME_FTA_H,LAST_8_GAME_FT_PCT_H,LAST_8_GAME_OREB_H,LAST_8_GAME_DREB_H,LAST_8_GAME_REB_H,LAST_8_GAME_AST_H,...,LAST_1_GAME_PLAYER_TM_TOV_PCT_A,LAST_1_GAME_PLAYER_EFG_PCT_A,LAST_1_GAME_PLAYER_TS_PCT_A,LAST_1_GAME_PLAYER_USG_PCT_A,LAST_1_GAME_PLAYER_PACE_A,LAST_1_GAME_PLAYER_PIE_A,LAST_2_GAME_PLAYER_MIN_A,LAST_2_GAME_PLAYER_OFF_RATING_A,LAST_2_GAME_PLAYER_DEF_RATING_A,LAST_2_GAME_PLAYER_NET_RATING_A,LAST_2_GAME_PLAYER_AST_PCT_A,LAST_2_GAME_PLAYER_AST_TOV_A,LAST_2_GAME_PLAYER_AST_RATIO_A,LAST_2_GAME_PLAYER_OREB_PCT_A,LAST_2_GAME_PLAYER_DREB_PCT_A,LAST_2_GAME_PLAYER_REB_PCT_A,LAST_2_GAME_PLAYER_TM_TOV_PCT_A,LAST_2_GAME_PLAYER_EFG_PCT_A,LAST_2_GAME_PLAYER_TS_PCT_A,LAST_2_GAME_PLAYER_USG_PCT_A,LAST_2_GAME_PLAYER_PACE_A,LAST_2_GAME_PLAYER_PIE_A,LAST_3_GAME_PLAYER_MIN_A,LAST_3_GAME_PLAYER_OFF_RATING_A,LAST_3_GAME_PLAYER_DEF_RATING_A,LAST_3_GAME_PLAYER_NET_RATING_A,LAST_3_GAME_PLAYER_AST_PCT_A,LAST_3_GAME_PLAYER_AST_TOV_A,LAST_3_GAME_PLAYER_AST_RATIO_A,LAST_3_GAME_PLAYER_OREB_PCT_A,LAST_3_GAME_PLAYER_DREB_PCT_A,LAST_3_GAME_PLAYER_REB_PCT_A,LAST_3_GAME_PLAYER_TM_TOV_PCT_A,LAST_3_GAME_PLAYER_EFG_PCT_A,LAST_3_GAME_PLAYER_TS_PCT_A,LAST_3_GAME_PLAYER_USG_PCT_A,LAST_3_GAME_PLAYER_PACE_A,LAST_3_GAME_PLAYER_PIE_A,LAST_4_GAME_PLAYER_MIN_A,LAST_4_GAME_PLAYER_OFF_RATING_A,LAST_4_GAME_PLAYER_DEF_RATING_A,LAST_4_GAME_PLAYER_NET_RATING_

# Lottery Data

In [ ]:
lottery_df = pd.read_pickle("LOTTERY_2012to2019.pkl")

In [ ]:
team = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")[['GAME_ID', 'TEAM_ID', 'GAME_DATE']].reset_index(drop=True)
team = team.merge(pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl")[['GAME_ID', 'TEAM_ID', 'GAME_DATE']].reset_index(drop=True),
                 on=["GAME_ID","GAME_DATE"],how="inner",suffixes=["_HOME","_AWAY"])

In [ ]:
team["GAME_DATE"] = team.GAME_DATE.apply(lambda dt : datetime.strptime(dt,"%Y-%m-%d").date())

In [ ]:
lottery_with_gameid = lottery_df.merge(team, on=["TEAM_ID_HOME","TEAM_ID_AWAY","GAME_DATE"], how="inner")

In [ ]:
lottery_df["GAME_DATE"] = lottery_df.GAME_DATE.apply(lambda dt : dt + timedelta(days=-1))
lottery_with_gameid = pd.concat([lottery_with_gameid,lottery_df.merge(team, on=["TEAM_ID_HOME","TEAM_ID_AWAY","GAME_DATE"], how="inner")],
                                axis=0).reset_index(drop=True)

In [ ]:
lottery_with_gameid = lottery_with_gameid.loc[:,[col for col in lottery_with_gameid.columns if col not in 
                                                 ['AWAY','HOME','TEAM_CH_NAME_HOME', 'TEAM_EN_NAME_HOME', 
                                                  'TEAM_CH_NAME_AWAY', 'TEAM_EN_NAME_AWAY','GAME_DATE']]]

In [ ]:
lottery_with_gameid.to_pickle("GAMEID_LOTTERY_2012to2019.pkl")

In [ ]:
lottery_df